# Finetuning a language model

Let's see how to finetune a language model to generate character backstories using HuggingFace Trainer with wandb integration. We'll use a tiny language model (`TinyStories-33M`) due to resource constraints, but the lessons you learn here should be applicable to large models too!

In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
import transformers
transformers.set_seed(42)

import wandb

/home/felix/llm/ChatTable/new_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: CUDA error: the launch timed out and was terminated
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
# in codespace environments wandb is unable to detect the nb name, that's why we need to set it manually
os.environ['WANDB_NOTEBOOK_NAME'] = '05_train_llm_starter.ipynb'

In [4]:
#wandb.login()
wandb.login(key="247b3da94c9b88bd5e990f1d94799ca3ded57d6b")

wandb: Currently logged in as: felix-ml (fml-team). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/felix/.netrc


True

In [5]:
model_checkpoint = "roneneldan/TinyStories-33M"

### Preparing data

We'll start by loading a dataset containing Dungeons and Dragons character biographies from Huggingface. 

> You can expect to get some warning here, this is ok

In [6]:
ds = load_dataset('MohamedRashad/characters_backstories')

In [7]:
# Let's take a look at one example
ds["train"][400]

{'text': 'Generate Backstory based on following information\nCharacter Name: Dewin \nCharacter Race: Halfling\nCharacter Class: Sorcerer bard\n\nOutput:\n',
 'target': 'Dewin thought he was a wizard, but it turned out it was the draconic blood in his veins that brought him eldritch power.  Music classes in wizarding college taught him yet another use for his power, and when he was expelled he took up adventuring'}

In [21]:
type(ds["train"][400]["text"])

str

In [8]:
# count the number of words in the dataset
total_words = 0
for example in ds["train"]:
    total_words += len(example["text"].split())
    total_words += len(example["target"].split())
print(f"Total number of words in the dataset: {total_words}")


Total number of words in the dataset: 594817


In [9]:
# As this dataset has no validation split, we will create one
ds = ds["train"].train_test_split(test_size=0.2, seed=42)

In [24]:
# We'll create a tokenizer from model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

# We'll need padding to have same length sequences in a batch
tokenizer.pad_token = tokenizer.eos_token

# Define a tokenization function that first concatenates text and target
def tokenize_function(example):
    merged = example["text"] + " " + example["target"]
    print(type(merged))
    print(merged)
    batch = tokenizer(merged, padding='max_length', truncation=True, max_length=128)
    batch["labels"] = batch["input_ids"].copy()
    return batch

# Apply it on our dataset, and remove the text columns
tokenized_datasets = ds.map(tokenize_function, remove_columns=["text", "target"])

Map:   4%|▎         | 68/1857 [00:00<00:02, 600.71 examples/s]

<class 'str'>
Generate Backstory based on following information
Character Name: Adriarin Melouchevine
Character Race: Wood elf
Character Class: Ranger

Output:
 Runs business side of shop, currently away on a mission to secure the Toern city watch contract. Arriving back to the fort just after the party's return. She was denied the contract due to unknown reasons (it was a Sending from Reynolds, who knows people in the city.)  Mama bear type. (Not actually a bear.)
<class 'str'>
Generate Backstory based on following information
Character Name: Keezle "Arch Jester Extraordinaire" Arcanodyne
Character Race: Gnome
Character Class: Wizard

Output:
At the moment Keezle enjoys tricking and annoying the evil population of the fallen kingdom of Mhourgh Brak. Currently his greatest enjoyment comes from the Young Adult Deep Dragon that dwells in a cave connected to the old Vault. very open to keezle creative ideas, at least in the beginning, it went out of hand, and Master Molton Runesprocket t

Map:  13%|█▎        | 238/1857 [00:00<00:02, 648.85 examples/s]

<class 'str'>
Generate Backstory based on following information
Character Name: Lisette Curtis
Character Race: Human
Character Class: Necromancer

Output:
 Lisette's church was destroyed by adventurers, leaving her disgraced in the eyes of her god.  She must bring the grasp of the tyrant's fist to the ocean through by becoming a pirate.
<class 'str'>
Generate Backstory based on following information
Character Name: Zuq Maddik
Character Race: Elf
Character Class: Monk

Output:
 Abandoned by his parents from birth, Zuq Kachiga knew nothing of his identity besides his father's whiskey bottle with his name engraved. Knowing nothing of his lineage, Zuq remained an outcasts throughout his adolescence. In response, the socially shunned elf changed his name, with rebellious spirit, to "Zuq Maddiq." Despite his pitiful upbringing, Zuq found success as a crime investigator and happily established a family life. This success, however, was short-lived. Briefly after his investigation work took dow

Map:  17%|█▋        | 322/1857 [00:00<00:02, 714.00 examples/s]

...

<class 'str'>
Generate Backstory based on following information
Character Name: Cormac
Character Race: Human
Character Class: Wizard

Output:
 Raised among the clans of the Axe Mountains, his career as a mercenary began once it was relatively clear that there was no room for a competitor in the sorcerous circles of his home. He endeavoured to join the Bloody Axe Mercenary Company as a way to see what greater wonders the world offers and to escape the petty concerns of the local chiefdoms. Since then, he has become a valuable member of the Company, fighting in countless war actions on land and sea.
<class 'str'>
Generate Backstory based on following information
Character Name: Clanless Kraygesh
Character Race: Dragonborn
Character Class: Cleric

Output:
Years later a vision told me of a great need of my skills and possibly my faith, although the details were vague. Four dragons seemed to be clear as day and a path leading north seemed to radiate in the dark for my eyes. A sign, I would 

Map:  75%|███████▌  | 351/465 [00:00<00:00, 654.58 examples/s]

<class 'str'>
Generate Backstory based on following information
Character Name: Theia
Character Race: Human 
Character Class: Cleric

Output:
 Theia was a street urchin who bacame a priestess after being caught stealing from a church. She was saved by a former paladin when her temple was attacked by bandits. The two were directed directed by visions from their goddess to travel to another continent to be her representatives on a council of cl
<class 'str'>
Generate Backstory based on following information
Character Name: Traminar
Character Race: Elf
Character Class: Rogue

Output:
 Born in an orphanage Traminar was raised by a group of wizards who taught him spells. His life taught him to be a rogue and how to fight as a warrior. His only weapon is an unbreakable wooden staff capped with diamonds on either end that can extend on a chain.
<class 'str'>
Generate Backstory based on following information
Character Name: Jimmy Hardtack
Character Race: Volksdeutsche
Character Class: Politica

Map: 100%|██████████| 465/465 [00:00<00:00, 658.00 examples/s]

<class 'str'>
Generate Backstory based on following information
Character Name: Larina Nix
Character Race: Human
Character Class: Witch

Output:
 Larina Nix was only six when she heard the call of the Goddess to become a witch.
<class 'str'>
Generate Backstory based on following information
Character Name: Bright Shield
Character Race: Human
Character Class: Kai lord

Output:
shield with its image.Ó it in glorious combat and emblazoning mymenceolk.
<class 'str'>
Generate Backstory based on following information
Character Name: Pith
Character Race: Elf (drow)
Character Class: Warlock

Output:
 Criminal
<class 'str'>
Generate Backstory based on following information
Character Name: Onan Bakkal
Character Race: Human
Character Class: Warlock

Output:
After leaving, he made his way to Duragzhar where he was able to put his warlock powers to use working in black market trades, smuggling, and treasure hunting. find the foul creature who killed his grandmother. He lost his way and was never se

In [22]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1857
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 465
    })
})

In [11]:
# Let's check out one prepared example
print(tokenizer.decode(tokenized_datasets["train"][900]['input_ids']))

Generate Backstory based on following information
Character Name: Mr. Gale
Character Race: Half-orc
Character Class: Cleric

Output:
 Growing up the only half-orc in a small rural town was rough. His mother didn't survive childbirth and so was raised in a church in a high mountain pass, his attention was always drawn by airships passing through, and dreams of an escape. Leaving to strike out on his own as early as he could he made a living for most of his life as an airship sailor, and occasionally a pirate. A single storm visits him throughout his life, marking every major


### Training
Let's finetune a pretrained language model on our dataset using HF Transformers and their wandb integration. 

In [12]:
# We will train a causal (autoregressive) language model from a pretrained checkpoint
model = AutoModelForCausalLM.from_pretrained(model_checkpoint);

In [13]:
# Start a new wandb run
run = wandb.init(project='fhtw_llm_tuning', job_type="training", anonymous="allow")

In [14]:
# write simple testscript to test torch, then transformers, then accelerate

# import torch
# import transformers
# from accelerate import Accelerator

# # Test torch
# x = torch.rand(5, 3)
# print(f"torch test passed. Tensor: \n{x}")

# # Test transformers
# model = transformers.BertModel.from_pretrained('bert-base-uncased')
# print(f"transformers test passed. Model architecture: \n{model}")

# # Test accelerate
# accelerator = Accelerator()
# print(f"accelerate test passed. Device: {accelerator.device}")

# print("All tests passed.")



In [15]:
import torch

# If CUDA is available, use it
if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device('cpu')


In [16]:
# Define training arguments
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-characters-backstories",
    report_to="wandb", # we need one line to track experiments in wandb
    num_train_epochs=1,
    logging_steps=1,
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    weight_decay=0.01,
    no_cuda=False, # force cpu use, will be renamed `use_cpu`
)

In [17]:
# We'll use HF Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

In [18]:
# Let's train!
trainer.train()

Epoch,Training Loss,Validation Loss


RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`

In [ ]:
transformers.logging.set_verbosity_error() # suppress tokenizer warnings

prefix = "Generate Backstory based on following information Character Name: "

prompts = [
    "Frogger Character Race: Aarakocra Character Class: Ranger Output: ",
    "Smarty Character Race: Aasimar Character Class: Cleric Output: ",
    "Volcano Character Race: Android Character Class: Paladin Output: ",
]

table = wandb.Table(columns=["prompt", "generation", "input_length", "output_length"])

for prompt in prompts:
    input_ids = tokenizer.encode(prefix + prompt, return_tensors="pt")
    input_ids = input_ids.to(device)
    output = model.generate(input_ids, do_sample=True, max_new_tokens=50, top_p=0.3)
    output_text = tokenizer.decode(output[0], skip_special_tokens=True)
    table.add_data(prefix + prompt, output_text, len(input_ids), len(output[0]))

    
wandb.log({'tiny_generations': table})

In [ ]:
wandb.finish()